$\newcommand{\To}{\Rightarrow}$
$\newcommand{\Suc}{\operatorname{Suc}}$
$\newcommand{\zero}{\mathrm{zero}}$

In [1]:
import os
os.chdir('..')

In [2]:
from kernel.type import NatType
from kernel.term import Var, Nat, Inst, Eq, Lambda
from kernel import theory
from kernel.proofterm import refl, ProofTerm
from logic import basic
from logic.conv import top_conv, rewr_conv, every_conv, try_conv, arg_conv
from logic.logic import apply_theorem
from data.nat import Suc, zero
from syntax.settings import settings

basic.load_theory('nat')
settings.unicode = True

## Peano arithmetic

In higher-order logic, types like natural numbers are defined as an inductive datatype, following the construction of natural numbers using Peano arithmetic. This construction specifies that the natural numbers can be built through two constructors: $\zero :: nat$ and $\Suc :: nat \To nat$ (here $\hbox{Suc}$ is short for successor). This is usually written as:

$$\begin{align*}
\hbox{datatype }nat = \zero\ |\ \Suc nat
\end{align*}$$

This means terms like $\zero, \Suc\zero, \Suc(\Suc \zero)$, etc are natural numbers. Moreover, these are the *only* natural numbers. It is clear that the numbers above correspond to $0, 1, 2$, etc.

The fact that these are the only natural numbers is reflected in the following theorem, called *principle of mathematical induction*:

In [3]:
theory.print_theorem('nat_induct')

nat_induct: ⊢ P 0 ⟶ (∀n. P n ⟶ P (Suc n)) ⟶ P x


The theorem says that, in order to prove any property $P$ holds for all natural numbers, it suffices to show that $P$ holds for $0$, and that if $P$ holds for $n$ then it holds for $\Suc n$ (that is, $n+1$). The reason we insist on using $\Suc n$ instead of $n+1$ is that so far, we are assuming that we have just defined natural numbers, and have not defined addition or proved any of its properties. This is what we will do next.

Addition on natural numbers is also defined by induction, more precisely, we define $m+n$ by induction on the first argument $m$. The definition consists of two equations, displayed as follows:

$$
\begin{align*}
& \mathbf{fun} \hbox{ plus} :: nat \To nat \To nat\ \mathbf{where} \\
& \quad 0 + n = n \\
& \quad \hbox{Suc } m + n = \hbox{Suc } (m + n)
\end{align*}
$$

The two equalities are given names `nat_plus_def_1` and `nat_plus_def_2` in holpy:

In [4]:
theory.print_theorem('nat_plus_def_1', 'nat_plus_def_2')

nat_plus_def_1: ⊢ 0 + n = n
nat_plus_def_2: ⊢ Suc m + n = Suc (m + n)


If we interpret $\Suc n$ as $n+1$, it is clear that these equations are true. Morever, these two equations suffice to compute the sum of any two natural numbers expressed using $\Suc$ and $\zero$. For example, the following computes $2+1=3$:

$$ \Suc(\Suc\zero) + \Suc\zero = \Suc(\Suc\zero + \Suc\zero) = \Suc(\Suc(\zero + \Suc\zero)) = \Suc(\Suc(\Suc\zero)) $$

Please check that the above calculation consists of two applications of `nat_plus_def_2`, followed by one application of `nat_plus_def_1`.

We can realize this in Python as follows:

In [5]:
norm_plus_cv = top_conv(every_conv(try_conv(rewr_conv('nat_plus_def_1')), try_conv(rewr_conv('nat_plus_def_2'))))

t = Suc(Suc(zero)) + Suc(zero)
print('t:', t)
print('eq:', refl(t).on_rhs(norm_plus_cv))

t: Suc (Suc 0) + Suc 0
eq: ProofTerm(⊢ Suc (Suc 0) + Suc 0 = Suc (Suc (Suc 0)))


Multiplication on natural numbers is defined in a similar way using induction:

$$
\begin{align*}
& \mathbf{fun} \hbox{ times} :: nat \To nat \To nat\ \mathbf{where} \\
& \quad 0 * n = 0 \\
& \quad \hbox{Suc } m * n = n + m * n
\end{align*}
$$

This is recorded as the following theorems:

In [6]:
theory.print_theorem('nat_times_def_1', 'nat_times_def_2')

nat_times_def_1: ⊢ 0 * n = 0
nat_times_def_2: ⊢ Suc m * n = n + m * n


We can compute with multiplication as follows:

In [7]:
norm_times_cv = top_conv(every_conv(try_conv(rewr_conv('nat_times_def_1')), try_conv(rewr_conv('nat_times_def_2'))))

t = Suc(Suc(zero)) * Suc(Suc(Suc(zero)))
print('t:', t)
print(refl(t).on_rhs(norm_times_cv))
print(refl(t).on_rhs(norm_times_cv, norm_plus_cv))

t: Suc (Suc 0) * Suc (Suc (Suc 0))
ProofTerm(⊢ Suc (Suc 0) * Suc (Suc (Suc 0)) = Suc (Suc (Suc 0)) + (Suc (Suc (Suc 0)) + 0))
ProofTerm(⊢ Suc (Suc 0) * Suc (Suc (Suc 0)) = Suc (Suc (Suc (Suc (Suc (Suc 0))))))


## Proofs using induction

We now give some examples of proofs using induction. First, note that while we know $0 + n = n$ directly from definition, the fact that $n + 0 = n$ still need to be proved. The proof is by induction on $n$. By induction, we need to prove $0+0=0$ and $\forall n.\,n+0=n \to \Suc n+0=\Suc n$.

#### Example:

Prove $n + 0 = n$.

#### Solution:

0. $\vdash 0 + 0 = 0$ by rewriting with nat_plus_def_1.
1. $n + 0 = n \vdash n + 0 = n$ by assume $n + 0 = n$.
2. $\vdash \Suc(n) + 0 = \Suc(n + 0)$ by rewriting with nat_plus_def_2.
3. $n + 0 = n \vdash \Suc(n) + 0 = \Suc(n)$ by rewriting 2 with 1.
4. $\vdash n + 0 = n \to \Suc(n) + 0 = \Suc(n)$ by implies_intr $n + 0 = n$ from 3.
5. $\vdash \forall n.\,n + 0 = n \to \Suc(n) + 0 = \Suc(n)$ by forall_intr $n$ from 4.
6. $n + 0 = n$ by apply_theorem nat_induct from 0, 5.

The proof can be realized in Python as follows:

In [8]:
n = Var('n', NatType)

pt0 = refl(Nat(0) + 0).on_rhs(rewr_conv('nat_plus_def_1'))
pt1 = ProofTerm.assume(Eq(n + 0, n))
pt2 = refl(Suc(n) + 0).on_rhs(rewr_conv('nat_plus_def_2'))
pt3 = pt2.on_rhs(arg_conv(rewr_conv(pt1)))
pt4 = pt3.implies_intr(Eq(n + 0, n))
pt5 = pt4.forall_intr(n)
pt6 = apply_theorem('nat_induct', pt0, pt5, inst=Inst(P=Lambda(n, Eq(n + 0, n)), x=n))
print(pt6)

ProofTerm(⊢ n + 0 = n)


The exported proof is as follows. It is a bit longer than the proof written above because of extra steps inserted by conversions.

In [9]:
print(pt6.export())

0: ⊢ 0 + ?n = ?n by theorem nat_plus_def_1
1: ⊢ (0::nat) + 0 = 0 by substitution {n: (0::nat)} from 0
2: ⊢ Suc ?m + ?n = Suc (?m + ?n) by theorem nat_plus_def_2
3: ⊢ Suc n + 0 = Suc (n + 0) by substitution {m: n, n: (0::nat)} from 2
4: ⊢ Suc = Suc by reflexive Suc
5: n + 0 = n ⊢ n + 0 = n by assume n + 0 = n
6: n + 0 = n ⊢ Suc (n + 0) = Suc n by combination from 4, 5
7: n + 0 = n ⊢ Suc n + 0 = Suc n by transitive from 3, 6
8: ⊢ n + 0 = n ⟶ Suc n + 0 = Suc n by implies_intr n + 0 = n from 7
9: ⊢ ∀n. n + 0 = n ⟶ Suc n + 0 = Suc n by forall_intr n from 8
10: ⊢ n + 0 = n by apply_theorem_for nat_induct, {P: λn::nat. n + 0 = n, x: n} from 1, 9


We do a second example, this time concerning multiplication.

#### Example:

Prove $n * 0 = 0$.

#### Solution:

0. $\vdash 0 * 0 = 0$ by rewriting with nat_times_def_1.
1. $n * 0 = 0 \vdash n * 0 = 0$ by assume $n * 0 = 0$.
2. $\vdash \Suc(n) * 0 = 0 + n * 0$ by rewriting with nat_times_def_2.
3. $\vdash \Suc(n) * 0 = n * 0$ by rewriting 2 with nat_plus_def_1.
4. $n * 0 = 0 \vdash \Suc(n) * 0 = 0$ by rewriting 3 with 1.
5. $\vdash n * 0 = 0 \to \Suc(n) * 0 = 0$ implies_intr $n * 0 = 0$ from 4.
6. $\vdash \forall n.\,n * 0 = 0 \to \Suc(n) * 0 = 0$ by forall_intr $n$ from 5.
7. $n * 0 = 0$ by apply_theorem nat_induct from 0, 6.

This is realized in Python as follows:

In [10]:
n = Var('n', NatType)

pt0 = refl(Nat(0) * 0).on_rhs(rewr_conv('nat_times_def_1'))
pt1 = ProofTerm.assume(Eq(n * 0, 0))
pt2 = refl(Suc(n) * 0).on_rhs(rewr_conv('nat_times_def_2'))
pt3 = pt2.on_rhs(rewr_conv('nat_plus_def_1'))
pt4 = pt3.on_rhs(rewr_conv(pt1))
pt5 = pt4.implies_intr(Eq(n * 0, 0))
pt6 = pt5.forall_intr(n)
pt7 = apply_theorem('nat_induct', pt0, pt6, inst=Inst(P=Lambda(n, Eq(n * 0, 0)), x=n))
print(pt2)

ProofTerm(⊢ Suc n * 0 = 0 + n * 0)


The same approach can be used to prove other properties of addition and multiplication, such as associativity, commutativity, and distributivity. We leave the proofs to later, when the use of tactics make them more convenient.